<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/FeatureDistancesNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calcul de la similitud entre imatges

Aquest fitxer segueix l'execució al fitxer de FeatureExtraction. Després d'haver guardat els vectors de característiques de les imatges pertanyens al conjunts d'entrenament i de test calcularem la seva similitud (distància) entre els vectors pertanyents a ambdós fitxers.

De cada vector pertanyent al conjunt de test cercarem els _k_ vectors del conjunt d'entrenament més propers i més llunyans. En el nostre cas _k_ = 5.

La distància obtinguda i la informació dels _k_ vectors més propers i llunyans es guardarà dins un fitxer de text. Degut al fet que en aquest projecte experimentarem amb les funcions L2 i cosin (per al càlcul de similitud), tindrem una carpeta per a cada una d'aquestes mètriques. Dins aquesta carpeta hi guardarem el nostre fitxer de text amb les distàncies obtingudes.

Les dues carpetes de L2 i cosin es guardaràn dins la carpeta 'Dist', que es situarà dins la mateixa carpeta dels pesos utilitzats en tot el procés.

Dins 'model/numero_execució/Dist/funció_distància/_'  hi crearem també una carpeta per a cada imatge de test. Aquestes carpetes tindrán el nom de l'identificador de la imatge amb la seva classe 'ISIC_00...(malaltia)'. Dins aquestes carpetes hi guardarem les _k_ imatges més properes i més llunyanes i la imatge de test en format jpg.

In [6]:
import torch
import cv2
import csv
import random
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import wandb
import pylab as pl
import json
import shutil

from glob import glob
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset
from google.colab import files
from IPython.display import clear_output,display
from google.colab.patches import cv2_imshow
from PIL import Image
from skimage import io
from google.colab import drive

DOWNLOAD = False

drive.mount('/content/drive')

MODELS = {
    0: 'AlexnetBinary',
    1: 'Resnet152Binary',
    2: 'Inceptionv3Binary',
    3: 'EfficientNetB1Binary',
    4: 'AlexnetMulticlass',
    5: 'Resnet152Multiclass',
    6: 'Inceptionv3Multiclass',
    7: 'EfficientnetB1Multiclass',
}

DISTANCES = {
    'Cosin' : 0,
    'L2' : 1
}


MODEL = MODELS[6] #triam un model
picked = 'L2'
DISTANCE = DISTANCES[picked] #triam una distància

RUN = 9 #triam el nombre de l'execució

#direcció dels fitxers de característiques
TRAINING_FEATURES_FILE = "/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/Features/training_features.txt"
TESTING_FEATURES_FILE = "/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/Features/testing_features.txt"

#cream el folder per a les distances
DIST_FOLDER = "/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/Dist/"+picked

if os.path.exists(DIST_FOLDER):
    shutil.rmtree(DIST_FOLDER)

os.makedirs(DIST_FOLDER,exist_ok=True)

if DOWNLOAD:
  !rm -rf /content/sample_data/*

  !kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

  !unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from io import TextIOWrapper
from numpy.linalg import norm
import sys
import ast
import shutil

def genera_dists():
  csv.field_size_limit(sys.maxsize)
  dict_illnesses = {0 : 'nv', 1 : 'mel', 2 : 'bkl', 3 : 'bcc', 4 : 'akiec', 5 : 'vasc', 6 : 'df'}

  fi = DIST_FOLDER+'/distances.txt'

  #informació guardada als fitxers de text
  headers = ["test_img","test_img_pred_label","test_img_label","top_distances","top_images_labels","top_images_names","bottom_distances","bottom_images_labels","bottom_images_names"]

  with open(fi, mode="w", newline="") as file:
      writer = csv.writer(file, delimiter=';')
      writer.writerow(headers)

  #obrim el fitxer de testing
  with open(TESTING_FEATURES_FILE ,newline = '') as ftst:

    test_features = csv.DictReader(ftst,delimiter=',')
    number = 0
    for i in test_features:
      number = number + 1
      test_img_name = i['Image Name']
      test_img_features = np.array(ast.literal_eval(i['FC Input']))
      test_img_path = i['Image Path']
      test_img_label = i['Label']

      if DISTANCE == 0: #si utilitzam la distància cosin, cercarem els valors més propers a 1
        top5dist = np.array([0,0,0,0,0],dtype=object)
        bottom5dist = np.array([np.inf,np.inf,np.inf,np.inf,np.inf],dtype=object)
      else: #d'altra banda, si utilitzam la distància l2, cercam els valors més propers a 0
        top5dist = np.array([np.inf,np.inf,np.inf,np.inf,np.inf],dtype=object)
        bottom5dist = np.array([0,0,0,0,0],dtype=object)

      top5labels = np.array([-1,-1,-1,-1,-1],dtype=object)
      bottom5labels = np.array([-1,-1,-1,-1,-1],dtype=object)
      top5names = np.array(["","","","",""],dtype=object)
      bottom5names = np.array(["","","","",""],dtype=object)
      top5paths = np.array(["","","","",""],dtype=object)
      bottom5paths = np.array(["","","","",""],dtype=object)

      #obrim el fitxer de training
      with open(TRAINING_FEATURES_FILE, newline='') as ftr:
        tr_features = csv.DictReader(ftr,delimiter = ',')

        for j in tr_features:
          train_img_name = j['Image Name']
          train_img_features = np.array(ast.literal_eval(j['FC Input']))
          train_img_path = j['Image Path']
          train_img_label = j['Label']

          if DISTANCE == 0: #cosin
            distance = np.dot(test_img_features,train_img_features)/(norm(test_img_features)*norm(train_img_features))
          else: #l2
            distance = norm(test_img_features - train_img_features)

          #actualitzam el top 5, bottom 5 o cap dels dos
          if DISTANCE == 0: #cosin
            for k in range(len(top5dist)):
              if distance > top5dist[k]:
                top5dist[k] = distance
                top5labels[k] = dict_illnesses[int(train_img_label)]
                top5paths[k] = train_img_path
                top5names[k] = train_img_name
                break
              if distance < bottom5dist[k]:
                bottom5dist[k] = distance
                bottom5labels[k] = dict_illnesses[int(train_img_label)]
                bottom5paths[k] = train_img_path
                bottom5names[k] = train_img_name
                break
          else: #l2
            for k in range(len(top5dist)):
              if distance < top5dist[k]:
                top5dist[k] = distance
                top5labels[k] = dict_illnesses[int(train_img_label)]
                top5paths[k] = train_img_path
                top5names[k] = train_img_name
                break
              if distance > bottom5dist[k]:
                bottom5dist[k] = distance
                bottom5labels[k] = dict_illnesses[int(train_img_label)]
                bottom5paths[k] = train_img_path
                bottom5names[k] = train_img_name
                break

        #we create the folder of our test image, it will have name and label
        os.makedirs(DIST_FOLDER+'/'+test_img_name.split('.')[0]+'('+dict_illnesses[int(test_img_label)]+')/',exist_ok=True)

        #we want to order the top and bottom depending on the distance
        if DISTANCE == 0:
          sorted_indices = np.argsort(top5dist)[::-1]
        else:
          sorted_indices = np.argsort(top5dist)

        #si utilitzam la distància cosin ens interessa que la distància més gran sigui la primera

        top5dist = top5dist[sorted_indices]
        top5labels = top5labels[sorted_indices]
        top5paths = top5paths[sorted_indices]
        top5names = top5names[sorted_indices]

        if DISTANCE == 0:
          sorted_indices = np.argsort(bottom5dist)[::-1]
        else:
          sorted_indices = np.argsort(bottom5dist)

        bottom5dist = bottom5dist[sorted_indices]
        bottom5labels = bottom5labels[sorted_indices]
        bottom5paths = bottom5paths[sorted_indices]
        bottom5names = bottom5names[sorted_indices]

        #corregir actual label
        data_row = [test_img_name.split('.')[0],test_img_label,test_img_label,[round(num,6) for num in top5dist.astype(float).tolist()],top5labels,top5names.astype(str).tolist(),[round(num,6) for num in bottom5dist.astype(float).tolist()],bottom5labels,bottom5names.astype(str).tolist()]

        #guardarem els data_row disn un fitxer de txt i crearem les carpetes top i bottom

        #ens interessa ara el nom de la enfermetat
        test_img_label = dict_illnesses[int(test_img_label)]

        os.makedirs(DIST_FOLDER+'/'+test_img_name.split('.')[0]+'('+test_img_label+')'+'/Top/',exist_ok=True)
        os.makedirs(DIST_FOLDER+'/'+test_img_name.split('.')[0]+'('+test_img_label+')'+'/Bottom/',exist_ok=True)

        #guardam les imatges més properes
        for l in range(len(top5paths)):
          shutil.copy(top5paths[l], DIST_FOLDER+'/'+test_img_name.split('.')[0]+'('+test_img_label+')/Top/'+str(l)+'_'+top5labels[l]+'.jpg')

        #guardam la nostra imatge de test
        shutil.copy(test_img_path ,DIST_FOLDER+'/'+test_img_name.split('.')[0]+'('+test_img_label+')/'+test_img_label+'_imatge_test_'+test_img_name)

        #guardam les imatges més llunyanes

        for m in range(len(bottom5paths)):
          shutil.copy(bottom5paths[m], DIST_FOLDER+'/'+test_img_name.split('.')[0]+'('+test_img_label+')/Bottom/'+str(m)+'_'+bottom5labels[m]+'.jpg')

        #guardam les distàncies dins un fitxer txt
        with open(fi, mode="a", newline="") as file:
          writer = csv.writer(file, delimiter=';')
          writer.writerow(data_row)  # Write row immediately

        print(number) #per fer un seguiment

        #Nombre de distàncies que volem guardar
        if number == 250:
          break

  print("CSV file written successfully.")

In [8]:
# !mkdir -p ~/.kaggle
# !mv /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


In [9]:
genera_dists()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
CSV file written successfully.
